In [77]:
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns

from model.preprocess import Preprocessor
from model.callbacks import SeqGenerateCallback
from model.model import Poet
from model.metrics import MaskedAccuracy, MaskedLoss

In [ ]:
MAX_VOCAB_SIZE = 10000
EMBEDDING_DIM = 200
DFF = 512
D_MODEL = 256
MAX_SEQ_LEN = 10

In [78]:
data = pd.read_csv("data.tsv", sep="\t")

In [79]:
data.head()

,lines
0,मैं तो केवल एक तथ्य हूं
1,सदियों से बदनाम हुआ हूं
2,किसी एक के नही सभी के
3,हाथों से मै छला गया हूं ।
4,"भला ,बुरा ,बीता और गुजरा"


In [80]:
train_data = data.iloc[:-5, :]
val_data = data.iloc[-5:, :]

In [81]:
train_data.shape

(15, 1)

In [82]:
preprocessor = Preprocessor(vocab_size=1000, seq_len=10)

In [83]:
vocab = preprocessor.build_vocab(train_data)
train_seq_data = preprocessor(train_data)
x_train_seq, y_train_seq = train_seq_data[:, :-1], train_seq_data[:, 1:]
val_seq_data = preprocessor(val_data)
x_val_seq, y_val_seq = val_seq_data[:, :-1], val_seq_data[:, 1:]

In [84]:
vocab[:20]

['',
 '[UNK]',
 '[SURU]',
 '[KHATAM]',
 'हूं',
 '।',
 'नहीं',
 'के',
 'किसी',
 'है',
 'हुआ',
 'से',
 'सुनता',
 'सदियों',
 'रहता',
 'ये',
 'भला',
 'और',
 'एक',
 'ही']

In [85]:
preprocessor.vocab_size

60

In [86]:
model = Poet(preprocessor=preprocessor, num_blocks=1, d_model=256, dff=512, heads=8, embedding_dims=100)
model.compile(loss=MaskedLoss(), optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), metrics=[MaskedAccuracy()])

In [87]:
trigger_inputs = [["मै"]]
trigger_inputs = preprocessor(trigger_inputs)
generator_callback = SeqGenerateCallback(trigger_inputs)

In [88]:
history = model.fit(x=x_train_seq, y=y_train_seq, batch_size=1, epochs=10, callbacks=[generator_callback])

Epoch 1/10


ValueError: in user code:

    C:\Users\tusha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tensorflow\python\keras\engine\training.py:855 train_function  *
        return step_function(self, iterator)
    d:\attention poet\model\model.py:113 call  *
        x = embeddings + self.pos_encoding
    C:\Users\tusha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tensorflow\python\ops\math_ops.py:1250 binary_op_wrapper
        raise e
    C:\Users\tusha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tensorflow\python\ops\math_ops.py:1234 binary_op_wrapper
        return func(x, y, name=name)
    C:\Users\tusha\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\tensorflow\python\util\dispatch.py:206 w